The following notebook supports the lecture on cryptography. The code is not designed to deliver optimal performance, but rather to illustrate the concepts in the clearest way. Minimal optimization has been done, with the exception of exponentiation as it could cause overflow errors. 

This notebook can be redistributed, adapted or copied freely, though the author should be acknowledged. This document may still contain typo’s / mistakes, and as such comments are welcome. Contact information can be found on the author’s website.

Seppe Staelens

In [1]:
import numpy as np

# Exercise

Solve $x^7 \equiv 5 \mod 143$

In [3]:
e = 7
n = 143
c = 5

for i in range(1,n + 1):
    LHS = i**e - c
    if LHS%n == 0:
        x = i
        print(x)
        break

125


In [4]:
print(x**e)
print(x**e - c)
print((x**e - c)/n)

476837158203125
476837158203120
3334525581840.0


# Example

## Functions

In [2]:
def chosen(p, q):
    """Calculate n and phi(n) from chosen p,q"""
    n = p*q
    phi_n = (p-1)*(q-1)
    print("n = ", n)
    print(r'phi(n) = ', phi_n)
    return n, phi_n

def check_e(e, phi_n):
    """Check if e is coprime to phi(n)"""
    if np.gcd(e, phi_n) == 1:
        print("e is coprime to phi(n)")
    else:
        print("e is not coprime to phi(n)")

def check_d(d, e, phi_n):
    """Check if d is the modular inverse of e mod phi(n)"""
    if d*e % phi_n == 1:
        print("d is the modular inverse of e mod phi(n)")
    else:
        print("d is not the modular inverse of e mod phi(n)")

def alt_ord(char):
    """Include space as 27th character in alphabet"""
    if char == ' ':
        return 27
    else:
        return ord(char) - 96
    
def alt_chr(num):
    """Include space as 27th character in alphabet"""
    if num == 27:
        return ' '
    else:
        return chr(num + 96)
    
def convert_message(message):
    """Convert message to numbers"""
    n_full_chunks = len(message) // 3
    len_partial_chunk = len(message) % 3

    print("Message length: ", len(message))
    print("Full chunks: ", n_full_chunks)
    print("Partial chunk: ", len_partial_chunk)

    Messages = []

    for i in range(n_full_chunks):
        chunk = message[i*3:i*3+3]
        print("Encrypting " + chunk)
        list = [alt_ord(char) for char in chunk.lower()]
        val = 28 * 100**3 + list[0] * 100**2 + list[1] * 100 + list[2]
        Messages.append(val)

    if len_partial_chunk > 0:
        chunk = message[-len_partial_chunk:]
        print("Encrypting " + chunk)
        val = 28 * 100**(len_partial_chunk)
        list = [alt_ord(char) for char in chunk.lower()]
        for i in range(len_partial_chunk):
            val += list[-len_partial_chunk + i] * 100**(len_partial_chunk - i - 1)
        Messages.append(val)

    return np.array(Messages)
    
def safe_exp(x, e, n):
    """Overflow safe modular exponentiation - 
    at least for small enough numbers"""
    result = 1
    for i in range(e):
        result = (result * x) % n
    return result

def num_to_let(M):
    """Translate numbers back to letters / space"""
    letters = ""
    while M > 28:
        letters += alt_chr(M % 100)
        M = M // 100
    return letters[::-1]

def decrypt(M):
    """Decrypt message"""
    message = ""
    for num in M:
        message += num_to_let(num)
    print(message)
    


## Example itself

In [12]:
# p and q secret
p = 3593
q = 8893
# p = 8761
# q = 5527

# n = pq and phi(n) = (p-1)(q-1)
# phi_n remains secret
# n becomes public
n, phi_n = chosen(p, q)

n =  31952549
phi(n) =  31940064


Note: for the code to work, $n > 29\: 000\: 000$

In [13]:
e = 5
# e = 7
check_e(e, phi_n)

e is coprime to phi(n)


In [14]:
# de = 1 mod phi(n)
d = 6388013
# d = 20746183
check_d(d, e, phi_n)

d is the modular inverse of e mod phi(n)


We will start our chunks of message with 28 in order to avoid leading zeroes when converting letters to numbers in the alphabet. This means that our message needs to be split in chunks of 3 letters, to form a number of the form
28.xx.xx.xx

In [15]:
message = "top secret info"

In [16]:
M = [alt_ord(char) for char in message.lower()]
print(M)

[20, 15, 16, 27, 19, 5, 3, 18, 5, 20, 27, 9, 14, 6, 15]


In [17]:
encoded = convert_message(message)
encoded

Message length:  15
Full chunks:  5
Partial chunk:  0
Encrypting top
Encrypting  se
Encrypting cre
Encrypting t i
Encrypting nfo


array([28201516, 28271905, 28031805, 28202709, 28140615])

In [18]:
encrypted = safe_exp(encoded, e, n)
encrypted

array([29886512, 13955083, 23895516,    42650, 17922868])

In [19]:
decrypt(encrypted)

}¸¡l¿³¹pz¼|¤


In [20]:
decrypted = safe_exp(encrypted, d, n)
print(decrypted)
decrypt(decrypted)

[28201516 28271905 28031805 28202709 28140615]
top secret info
